In [15]:
from matplotlib import animation
from matplotlib import pyplot as plt

from CrankNicholsonSolver import Solver

import numpy as np
import matplotlib

matplotlib.use("TkAgg")

![title](Task.png)

## Вспомогательные функции для анимации

In [16]:
def AnimateSolution2D(xx: np.array, yy: np.array, zz: list):
    def update_plot_2D(frame):
        nonlocal plot
        plot.remove()
        plot = plot = ax.pcolormesh(xx, yy, zz[frame], cmap='magma', vmin=0, vmax=1)
        return plot,

    fig = plt.figure(figsize=(100, 100), dpi=200)
    ax = plt.axes()
    plot = ax.pcolormesh(xx, yy, zz[0])
    fig.colorbar(plot, ax=ax)
    ax.set_title('2D Heat map')
    anim = animation.FuncAnimation(fig, update_plot_2D, len(zz), interval=30)
    plt.show()


def AnimateSolution3D(xx: np.array, yy: np.array, zz: list):
    def update_plot(frame):
        nonlocal plot
        plot.remove()
        plot = ax.plot_surface(xx, yy, zz[frame], cmap='magma', vmin=0, vmax=1)
        return plot,

    fig = plt.figure(figsize=(100, 100), dpi=200)
    ax = plt.axes(projection='3d')
    ax.set_zlim(0, 1)
    plot = ax.plot_surface(xx, yy, zz[0])
    fig.colorbar(plot, ax=ax)
    ax.set_title('Heat map')
    anim = animation.FuncAnimation(fig, update_plot, len(zz), interval=30)
    plt.show()

## Интенсивность источника тепла
$$ƒ(x, y, t) = \left( 1 - \frac{x^{2}}{L^{2}} \right)*\left( 1 - \frac{y^{2}}{L^{2}} \right)$$

In [17]:
def f(x, y):
    return (1 - x ** 2 / L ** 2) * (1 - y ** 2 / L ** 2)

## Параметры задачи
 $L = 1; \ T = 1; \ dt = 0.01 $

In [18]:
L = 1

dt = 0.001
T = 1

nx = 1000
ny = 1000
x0, x1, y0, y1 = (-L, L, -L, L)

## Решение уравнения локально-одномерным методом
На каждом шаге по времени отдельно рассматриваем распространение тепла по $x$ и по $y$.
В качестве схемы используем схему Кранка-Николсона

In [19]:
%%time
x_solvers = [Solver(x0, x1, nx, dt / 2) for _ in range(ny)]
y_solvers = [Solver(y0, y1, ny, dt / 2) for _ in range(nx)]

X, Y = np.meshgrid(x_solvers[0].xs, y_solvers[0].xs)
frames = []
solution = np.vectorize(f, otypes=[float])(X, Y)

for i in range(int(T / dt)):
    frames.append(np.copy(solution))

    for j, s in enumerate(x_solvers):
        solution[j + 1, :] = s.do_step(solution[j + 1, :])
    for j, s in enumerate(y_solvers):
        solution[:, j + 1] = s.do_step(solution[:, j + 1])

CPU times: user 4min 43s, sys: 2.26 s, total: 4min 45s
Wall time: 4min 45s


### Распространение тепла в 3D

In [20]:
AnimateSolution3D(X, Y, frames)

### Распространение тепла в 2D

In [21]:
AnimateSolution2D(X, Y, frames)